### SCI 期刊名称提取

In [17]:
import re
import pandas as pd

with open('other/SSCI-MANAGEMENT.txt','r') as f:
    data = f.read()
len(data)

53168

In [18]:
Magazines = re.findall('\n\d+.(.*?)\n',data)
len(Magazines)

217

In [19]:
ISSN = re.findall('\nISSN: (.*?)\n',data)
len(ISSN)

217

In [20]:
temp = []
for i in range(len(Magazines)):
    row = {'期刊名':Magazines[i].strip(),'ISSN':ISSN[i].strip()}
    temp.append(row)
df = pd.DataFrame(temp)
df.to_excel('SSCI-MANAGEMENT-2019目录-官网.xlsx',index=False)

### 高校中英文名称对照

In [2]:
import pandas as pd
with open('高校中英文名称对照.txt','r',encoding='utf-8') as f:
    data = f.readlines()
temp = []
for item in data:
    name_cn = item.split('   ')[0].strip()
    name_en = item.split('   ')[1].strip()
    row = {'高校中文名称':name_cn,'高校英文名称':name_en}
    temp.append(row)

df = pd.DataFrame(temp)
df.to_excel('高校中英文名称对照.xlsx')

### 万方-外文期刊采集

In [2]:
import pandas as pd
from time import sleep
import re
import threading
import random
from selenium import webdriver # 从selenium导入webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import os
from itertools import product

options = webdriver.ChromeOptions()
with open('工商管理/succeed.txt','r',encoding='utf-8') as f:
    data = f.readlines()
succeed = [x.strip() for x in data] #记录完成的名单
mylock = threading.Lock() #给下载成功的名单加锁
def get_info(colleges,magazines):
    global options
    global succeed
    #用笛卡尔积替代双重for循环，稳
    random.shuffle(colleges) #随机打乱
    random.shuffle(magazines)
    user_agent_list = [ \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60" \
        "Opera/8.0 (Windows NT 5.1; U; en)"\
        "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0" \
        "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER" \
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0" \
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)" \
    ]
    url = 'http://www.wanfangdata.com.cn/searchResult/getAdvancedSearch.do?searchType=all'
    for college,magazine in product(colleges,magazines):
        failed_count = 0 #用于记录失败次数，暂时休息应对反爬
        the_item = college+'-'+magazine
        if the_item in succeed:
            #print(the_item+' 已完成.')
            continue
        isExists = os.path.exists('工商管理' + '/' + college) #创建学者文件夹
        if not isExists:
            os.makedirs('工商管理' + '/' + college)
        UserAgent = 'user-agent=' + random.choice(user_agent_list)
        options.add_argument(UserAgent)
        index_path = 'D:\\codes\\GitProject\\工作区\\首经贸数据采集项目\\论文数据\\外文期刊\\工商管理'
        college_path = index_path + '\\' + college
        prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': college_path}  #下载路径
        options.add_experimental_option('prefs', prefs)
        driver = webdriver.Chrome(chrome_options=options)
        try:  #访问
            driver.get(url)
        except:
            try:
                driver.refresh()
            except:
                print('超时错误')
                driver.quit()
                continue
        sleep(2)
        try:
            driver.find_element_by_id('cancel_all').click() #清除所有
            driver.find_elements_by_name('thesis_type')[0].click() #期刊论文

            Select(driver.find_elements_by_name('search_field')[1]).select_by_value('ISSN/CN')
            mag = driver.find_elements_by_id('ddd')[1]   
            mag.send_keys(magazine)
            Select(driver.find_elements_by_name('vague_accurate')[1]).select_by_value('accurate')#精确查找
            col = driver.find_elements_by_id('ddd')[2]
            col.send_keys(college)
            search = driver.find_element_by_id('set_advanced_search_btn')
            search.click() #点击查找
            sleep(8)
            page = driver.page_source
            if page.find('没有命中的记录，请重试或修改检索表达式') > -1:
                print(the_item + '  没有检索结果')
                succeed.append(the_item)
                driver.quit()
                sleep(3)
                continue

            while True:
                driver.find_element_by_name('checkAll').click() #全选
                sleep(3)
                try:
                    driver.find_element_by_link_text(u'下一页').click()
                    sleep(9)
                except:
                    #print('没有下一页')
                    sleep(1)
                    break
            #导出前进行判断，是否把所有的检索结果都选中了
            result = driver.find_element_by_class_name('result_stati').text
            m1 = re.search('\d+',result)
            result_counts = m1.group(0)
            my_result = driver.find_element_by_name('checkAll').text
            m2 = re.search('\d+',my_result)
            my_counts = m2.group(0)
            if result_counts != my_counts:
                print(scholar +' 检索结果:' + str(result_counts) + ' 选中结果:' + str(my_counts))
                print('检索未全部选中，本次作废')
                driver.quit()
                sleep(4)
                continue
            #批量导出
            driver.find_element_by_link_text(u'批量导出').click()
            driver.switch_to_window(driver.window_handles[1]) #可能要重新定位windows
            loc2 = (By.ID,'export')
            ele2 = WebDriverWait(driver,10).until(EC.presence_of_element_located(loc2))
            driver.find_element_by_link_text(u'自定义格式').click()
            sleep(5)
            driver.find_element_by_class_name('export_excel').click()
            print( the_item + ' 抓取成功')
            sleep(15)
            succeed.append(the_item)
            succeed = list(set(succeed))
            with open('工商管理/succeed.txt','w',encoding='utf-8') as f:
                for item in succeed:
                    f.write(item)
                    f.write('\n')
            if len(set(succeed)) == 2170: #工商管理-16*38 
                print('工商管理-外文期刊全部下载完毕')
                break
            driver.quit()
        except:
            print(the_item+' 抓取失败')
            failed_count += 1
            if failed_count == 3: #如果连续3次失败，则跳出循环
                driver.quit()
                sleep(150)
            else:
                sleep(5)
                driver.quit()
                
def to_download(counts):
    data = pd.read_excel('高校中英文名称对照.xlsx',sheet_name='管理学')
    colleges = data['高校英文名称']
    data = pd.read_excel('Magazines.xlsx',sheet_name='管理学')
    magazines = data['ISSN']
    for i in range(counts):
        print('第'+str(i+1)+'次循环')
        get_info(colleges,magazines)
        
if __name__ == '__main__':
    counts = 3
    threads = []    
    for i in range(4):
        t = threading.Thread(target=to_download,args=(counts,))
        threads.append(t)
    for t in threads:
        t.start()
    for thread in threads:
        thread.join()

第1次循环
第1次循环
第1次循环
第1次循环
Zhejiang Gongshang University-1740-4754  没有检索结果
Jiangxi University of Finance and Economics-0965-3562  没有检索结果
Tianjin University of Finance and Economics-0964-8410  没有检索结果
Tianjin University of Finance and Economics-1617-9846  没有检索结果
Zhejiang Gongshang University-1058-6407  没有检索结果
Jiangxi University of Finance and Economics-0969-6016  没有检索结果
Shanxi University Of Finance And Economics-0266-2426 抓取失败
Tianjin University of Finance and Economics-1740-4754  没有检索结果
Zhejiang Gongshang University-2042-5791  没有检索结果
Jiangxi University of Finance and Economics-1480-8986  没有检索结果
Shanxi University Of Finance And Economics-0960-6491  没有检索结果
Tianjin University of Finance and Economics-1350-5084  没有检索结果
Zhejiang Gongshang University-1046-4964  没有检索结果
Jiangxi University of Finance and Economics-0008-1256  没有检索结果
Shanxi University Of Finance And Economics-0160-8061  没有检索结果
Zhejiang Gongshang University-1936-8623  没有检索结果
Shanxi University Of Finance And Economics-2040-8021  没有检索结果

Jiangxi University of Finance and Economics-0277-6693  没有检索结果
Shanxi University Of Finance And Economics-1366-2716  没有检索结果
Zhejiang Gongshang University-0963-1690  没有检索结果
Tianjin University of Finance and Economics-0378-7206  没有检索结果
Jiangxi University of Finance and Economics-1750-4708  没有检索结果
Shanxi University Of Finance And Economics-0268-3962  没有检索结果
Zhejiang Gongshang University-2210-5395  没有检索结果
Tianjin University of Finance and Economics-1476-7503  没有检索结果
Jiangxi University of Finance and Economics-1441-3523  没有检索结果
Shanxi University Of Finance And Economics-1350-231X  没有检索结果
Zhejiang Gongshang University-1355-2554  没有检索结果
Tianjin University of Finance and Economics-2222-3436  没有检索结果
Jiangxi University of Finance and Economics-0170-8406  没有检索结果
Shanxi University Of Finance And Economics-1756-6517  没有检索结果
Zhejiang Gongshang University-0956-5221  没有检索结果
Tianjin University of Finance and Economics-0272-6963  没有检索结果
Shanxi University Of Finance And Economics-1740-8776  没有检索结果
Jiangxi

Shanxi University Of Finance And Economics-1806-4892  没有检索结果
Zhejiang Gongshang University-0013-791X  没有检索结果
Jiangxi University of Finance and Economics-0966-0879  没有检索结果
Shanxi University Of Finance And Economics-1048-6682  没有检索结果
Tianjin University of Finance and Economics-1012-8255  没有检索结果
Zhejiang Gongshang University-0883-7066  没有检索结果
Jiangxi University of Finance and Economics-0966-0429  没有检索结果
Tianjin University of Finance and Economics-0041-1612  没有检索结果
Shanxi University Of Finance And Economics-0166-4972  没有检索结果
Zhejiang Gongshang University-1746-5265  没有检索结果
Jiangxi University of Finance and Economics-0959-0552  没有检索结果
Tianjin University of Finance and Economics-0019-8501  没有检索结果
Shanxi University Of Finance And Economics-1537-260X  没有检索结果
Jiangxi University of Finance and Economics-0971-7218  没有检索结果
Tianjin University of Finance and Economics-1537-260X  没有检索结果
Zhejiang Gongshang University-1367-5567 抓取成功
Shanxi University Of Finance And Economics-1367-3270  没有检索结果
Jiangxi Un

Jiangxi University of Finance and Economics-1757-5818  没有检索结果
Tianjin University of Finance and Economics-0264-2069  没有检索结果
Zhejiang Gongshang University-1212-3609  没有检索结果
Shanxi University Of Finance And Economics-1548-0518  没有检索结果
Jiangxi University of Finance and Economics-2059-5794  没有检索结果
Tianjin University of Finance and Economics-0894-3796  没有检索结果
Zhejiang Gongshang University-0001-4273  没有检索结果
Shanxi University Of Finance And Economics-0031-5826  没有检索结果
Jiangxi University of Finance and Economics-0048-3486  没有检索结果
Tianjin University of Finance and Economics-0030-364X  没有检索结果
Zhejiang Gongshang University-0959-6119  没有检索结果
Shanxi University Of Finance And Economics-2078-5585  没有检索结果
Jiangxi University of Finance and Economics-1046-4964  没有检索结果
Tianjin University of Finance and Economics-1047-7047  没有检索结果
Zhejiang Gongshang University-1045-3172  没有检索结果
Shanxi University Of Finance And Economics-0047-2506  没有检索结果
Jiangxi University of Finance and Economics-0021-9010  没有检索结果
Tianji

Zhejiang Gongshang University-0889-938X  没有检索结果
Shanxi University Of Finance And Economics-1480-8986  没有检索结果
Jiangxi University of Finance and Economics-1744-9359  没有检索结果
Tianjin University of Finance and Economics-1475-9551  没有检索结果
Zhejiang Gongshang University-1740-8776  没有检索结果
Shanxi University Of Finance And Economics-1045-3172  没有检索结果
Jiangxi University of Finance and Economics-1806-4892  没有检索结果
Tianjin University of Finance and Economics-1582-9146  没有检索结果
Zhejiang Gongshang University-1350-5084  没有检索结果
Shanxi University Of Finance And Economics-0956-5221  没有检索结果
Jiangxi University of Finance and Economics-1047-7039  没有检索结果
Tianjin University of Finance and Economics-1740-8776  没有检索结果
Zhejiang Gongshang University-1833-3672  没有检索结果
Shanxi University Of Finance And Economics-1362-0436  没有检索结果
Jiangxi University of Finance and Economics-2212-571X  没有检索结果
Tianjin University of Finance and Economics-0022-2380  没有检索结果
Shanxi University Of Finance And Economics-0949-6181  没有检索结果
Zhejian

...

Zhongnan University of Economics and Law-0894-3796  没有检索结果
Capital University of Economics and Business-2041-3866  没有检索结果
Dongbei University of Finance and Economics-1477-8238  没有检索结果
Zhongnan University of Economics and Law-0927-7099  没有检索结果
Capital University of Economics and Business-1361-2026  没有检索结果
Southwestern University of Finance and Economics-0261-5177 抓取成功
Zhongnan University of Economics and Law-1350-5084  没有检索结果
Dongbei University of Finance and Economics-0160-5682 抓取成功
Capital University of Economics and Business-0092-2102  没有检索结果
Zhongnan University of Economics and Law-0017-8012  没有检索结果
Capital University of Economics and Business-0938-8249  没有检索结果
Southwestern University of Finance and Economics-0019-8501 抓取成功
Dongbei University of Finance and Economics-1075-4253  没有检索结果
Zhongnan University of Economics and Law-1012-8255  没有检索结果
Capital University of Economics and Business-1058-6407  没有检索结果
Dongbei University of Finance and Economics-1367-3270  没有检索结果
Zhongnan Universi

Zhongnan University of Economics and Law-1044-8004  没有检索结果
Zhongnan University of Economics and Law-1475-9551  没有检索结果
Dongbei University of Finance and Economics-1751-6757  没有检索结果
Capital University of Economics and Business-1463-7154  没有检索结果
Zhongnan University of Economics and Law-0957-4093  没有检索结果
Zhongnan University of Economics and Law-0958-5192  没有检索结果
Dongbei University of Finance and Economics-1746-8809  没有检索结果
Capital University of Economics and Business-0217-4561  没有检索结果
Zhongnan University of Economics and Law-0953-7325  没有检索结果
Zhongnan University of Economics and Law-0954-5395  没有检索结果
Dongbei University of Finance and Economics-1478-4092  没有检索结果
Capital University of Economics and Business-0142-5455  没有检索结果
Zhongnan University of Economics and Law-0272-6963  没有检索结果
Zhongnan University of Economics and Law-0883-7066  没有检索结果
Dongbei University of Finance and Economics-1350-5076  没有检索结果
Capital University of Economics and Business-0954-5395  没有检索结果
Zhongnan University of Econo

Zhongnan University of Economics and Law-0959-6119  没有检索结果
Capital University of Economics and Business-0968-6673  没有检索结果
Dongbei University of Finance and Economics-0748-4526  没有检索结果
Zhongnan University of Economics and Law-0735-3766  没有检索结果
Capital University of Economics and Business-1471-9037  没有检索结果
Zhongnan University of Economics and Law-0038-0121 抓取失败
Dongbei University of Finance and Economics-0277-6693  没有检索结果
Zhongnan University of Economics and Law-1053-4822  没有检索结果
Capital University of Economics and Business-0953-7325  没有检索结果
Zhongnan University of Economics and Law-0969-9988  没有检索结果
Dongbei University of Finance and Economics-0092-2102  没有检索结果
Zhongnan University of Economics and Law-0018-9391  没有检索结果
Capital University of Economics and Business-0160-5682  没有检索结果
Zhongnan University of Economics and Law-1478-4092  没有检索结果
Dongbei University of Finance and Economics-0008-1256  没有检索结果
Zhongnan University of Economics and Law-1757-5818  没有检索结果
Capital University of Economics

Zhongnan University of Economics and Law-1648-715X 抓取成功
Capital University of Economics and Business-0143-6570 抓取成功
Zhongnan University of Economics and Law-1540-1960  没有检索结果
Dongbei University of Finance and Economics-1441-3523  没有检索结果
Zhongnan University of Economics and Law-1750-614X  没有检索结果
Zhongnan University of Economics and Law-1746-8809  没有检索结果
Dongbei University of Finance and Economics-0267-257X  没有检索结果
Capital University of Economics and Business-0893-3189  没有检索结果
Dongbei University of Finance and Economics-0011-7315  没有检索结果
Zhongnan University of Economics and Law-0025-1747  没有检索结果
Capital University of Economics and Business-0170-8406  没有检索结果
Capital University of Economics and Business-0737-6782 抓取失败
Dongbei University of Finance and Economics-1877-8585  没有检索结果
Zhongnan University of Economics and Law-2397-0022  没有检索结果
Capital University of Economics and Business-1523-2409  没有检索结果
Dongbei University of Finance and Economics-1447-9338  没有检索结果
Zhongnan University of Economi

Dongbei University of Finance and Economics-1478-3363  没有检索结果
Capital University of Economics and Business-1212-3609  没有检索结果
Capital University of Economics and Business-1806-4892  没有检索结果
Dongbei University of Finance and Economics-0031-5826  没有检索结果
Dongbei University of Finance and Economics-0958-5192  没有检索结果
Capital University of Economics and Business-0963-1690  没有检索结果
Dongbei University of Finance and Economics-2041-3866  没有检索结果
Dongbei University of Finance and Economics-0160-8061  没有检索结果
超时错误
Capital University of Economics and Business-2055-6225  没有检索结果
Dongbei University of Finance and Economics-1086-0266  没有检索结果
Dongbei University of Finance and Economics-1038-4111  没有检索结果
Zhejiang Gongshang University-1554-7191  没有检索结果
Dongbei University of Finance and Economics-0895-6308  没有检索结果
Dongbei University of Finance and Economics-0888-4773  没有检索结果
Capital University of Economics and Business-1053-4822  没有检索结果
Dongbei University of Finance and Economics-0960-6491  没有检索结果
Dongbei Univ